# Milestone 3

In [2]:
%matplotlib inline

import os
import sys
import nltk
import json
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from unidecode import unidecode
from nltk.corpus import stopwords
from collections import OrderedDict
from matplotlib import pyplot as plt
from sklearn.naive_bayes import MultinomialNB
#from sklearn.model_selection import cross_val_score

plt.style.use('ggplot')

## Importing the data

In [113]:
raw = pd.read_json('data/themoviedb-5k-sample-4-15-2017.json')
print(raw.shape)
raw.head()

(5000, 29)


,adult,backdrop_path,belongs_to_collection,budget,cast,crew,genre_ids,genres,homepage,id,...,revenue,reviews,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,None,None,0,[],"[{u'name': u'Bobby Sheehan', u'department': u'...",[99],"[{u'name': u'Documentary', u'id': 99}]",https://www.boughtmovie.net,328380,...,0,[],93.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Bought,False,0.0,0
1,False,None,None,0,[],"[{u'name': u'Nancy Donnelly', u'department': u...",[99],"[{u'id': 99, u'name': u'Documentary'}]",,112052,...,0,[],60.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Egypt Underworld,False,3.8,2
2,False,/jWgtb7CRy3jWoXiz0IK352MG8b5.jpg,None,0,"[{u'name': u'Zhang Jin', u'character': u'Chau ...","[{u'name': u'Douglas Kung Cheung-Tak', u'depar...",[28],"[{u'id': 28, u'name': u'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706,...,0,[],1.0,"[{u'name': u'广州话 / 廣州話', u'iso_639_1': u'cn'},...",Released,,Chinese Heroes,False,0.0,0
3,False,None,None,0,"[{u'name': u'Erika Eleniak', u'character': u'J...","[{u'name': u'Brian Katkin', u'department': u'D...","[28, 18, 53]","[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",,61803,...,0,[],92.0,"[{u'name': u'English', u'iso_639_1': u'en'}]",Released,he quake of the century... get ready to rumble...,Shakedown,False,3.0,2
4,False,/sHH63HMAQsKLl0vpJ6AxulmkL92.jpg,None,0,"[{u'name': u'Kay Panabaker', u'character': u'J...","[{u'name': u'Kevin Tancharoen', u'department':...","[10402, 35, 18, 10749]","[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",,28665,...,0,[],107.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Fame,False,5.4,109


## Converting genres to bigrams

In [123]:
all_bigrams_ever = []

def bigrams(row):
    return zip(row['genre_lists'], row['genre_lists'][1:])

def col_for_each_bigram(row):
    for bigram in row['genre_bigrams']:            
        all_bigrams_ever.append(tuple(sorted(bigram)))
    return row

def popular_bigram(row):
    for k in row['genre_bigrams']:
        #if bigram_frequency[k] <= 50 and len(row['genre_lists']) != 1:
            #row['popular_bigram'] = 'Other'
        if k in OrderedDict(bigram_frequency.most_common()).keys() and row['popular_bigram'] != k:
            row['popular_bigram'] = k
    if len(row['genre_lists']) == 1:
        row['popular_bigram'] = tuple(row['genre_lists'])
    return row
                
raw['genre_lists'] = raw.apply(lambda x: [y['name'] for y in x['genres']], axis = 1)
for i in range(raw.shape[0]):
    raw.loc[i, 'genre_lists'].sort()

raw['genre_bigrams'] = raw.apply(lambda x: bigrams(x), axis = 1)
raw['popular_bigram'] = np.nan
raw = raw.apply(lambda x: col_for_each_bigram(x), axis = 1)
bigram_frequency = Counter(all_bigrams_ever)

raw = raw.apply(lambda x: popular_bigram(x), axis = 1)
raw.head(15)

,adult,backdrop_path,belongs_to_collection,budget,cast,crew,genre_ids,genres,homepage,id,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_lists,genre_bigrams,popular_bigram
0,False,None,None,0,[],"[{u'name': u'Bobby Sheehan', u'department': u'...",[99],"[{u'name': u'Documentary', u'id': 99}]",https://www.boughtmovie.net,328380,...,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Bought,False,0.0,0,[Documentary],[],"(Documentary,)"
1,False,None,None,0,[],"[{u'name': u'Nancy Donnelly', u'department': u...",[99],"[{u'id': 99, u'name': u'Documentary'}]",,112052,...,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Egypt Underworld,False,3.8,2,[Documentary],[],"(Documentary,)"
2,False,/jWgtb7CRy3jWoXiz0IK352MG8b5.jpg,None,0,"[{u'name': u'Zhang Jin', u'character': u'Chau ...","[{u'name': u'Douglas Kung Cheung-Tak', u'depar...",[28],"[{u'id': 28, u'name': u'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706,...,"[{u'name': u'广州话 / 廣州話', u'iso_639_1': u'cn'},...",Released,,Chinese Heroes,False,0.0,0,[Action],[],"(Action,)"
3,False,None,None,0,"[{u'name': u'Erika Eleniak', u'character': u'J...","[{u'name': u'Brian Katkin', u'department': u'D...","[28, 18, 53]","[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",,61803,...,"[{u'name': u'English', u'iso_639_1': u'en'}]",Released,he quake of the century... get ready to rumble...,Shakedown,False,3.0,2,"[Action, Drama, Thriller]","[(Action, Drama), (Drama, Thriller)]","(Drama, Thriller)"
4,False,/sHH63HMAQsKLl0vpJ6AxulmkL92.jpg,None,0,"[{u'name': u'Kay Panabaker', u'character': u'J...","[{u'name': u'Kevin Tancharoen', u'department':...","[10402, 35, 18, 10749]","[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",,28665,...,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Fame,False,5.4,109,"[Comedy, Drama, Music, Romance]","[(Comedy, Drama), (Drama, Music), (Music, Roma...","(Music, Romance)"
5,False,/tof5wSxZR4yoeBRo28kHEf8Uz3S.jpg,None,0,"[{u'name': u'John Lloyd Cruz', u'character': u...","[{u'name': u'Cathy Garcia-Molina', u'departmen...","[35, 10749]","[{u'id': 35, u'name': u'Comedy'}, {u'id': 1074...",,53236,...,"[{u'iso_639_1': u'en', u'name': u'English'}, {...",Released,,My Amnesia Girl,False,5.8,3,"[Comedy, Romance]","[(Comedy, Romance)]","(Comedy, Romance)"
6,False,None,None,0,"[{u'name': u'Chris Thombs', u'character': u'Th...",[],[28],"[{u'id': 28, u'name': u'Action'}]",http://sixaces.decemberized.com/,87601,...,"[{u'name': u'English', u'iso_639_1': u'en'}]",Released,,Six Aces,False,0.0,0,[Action],[],"(Action,)"
7,False,/v2KrZAgVqEvnHXm3LAu90VM7qOf.jpg,None,0,"[{u'name': u'Madness', u'character': u'Themsel...","[{u'name': u'Madness', u'department': u'Writin...",[10402],"[{u'id': 10402, u'name': u'Music'}]",http://www.blog.madness.co.uk,124184,...,"[{u'name': u'English', u'iso_639_1': u'en'}]",Released,,Madness at Madstock,False,8.0,1,[Music],[],"(Music,)"
8,False,/qDfq9RSEg5iAaJ02uax8qZPs7PX.jpg,None,0,"[{u'name': u'Andy García', u'character': u'Art...","[{u'name': u'Joseph Sargent', u'department': u...","[99, 18]","[{u'id': 99, u'name': u'Documentary'}, {u'id':...",,33355,...,"[{u'name': u'Español', u'iso_639_1': u'es'}, {...",Released,,For Love or Country: The Arturo Sandoval Story,False,4.0,1,"[Documentary, Drama]","[(Documentary, Drama)]","(Documentary, Drama)"
9,False,/weHp8DdMLw8vSJgafGfOLdD5mtw.jpg,None,0,"[{u'name': u'Simon Srebnik', u'character': u'H...","[{u'name': u'Claude Lanzmann', u'department': ...",[99],"[{u'id': 99, u'name': u'Documentary'}]",,42044,...,"[{u'name': u'English', u'iso_639_1': u'en'}, {...",Released,,Shoah,False,8.5,31,[Documentary],[],"(Documentary,)"


In [124]:
genre_frequency = raw['popular_bigram'].value_counts()
genre_frequency

(Documentary,)                 667
(Drama,)                       462
(Comedy,)                      347
(Drama, Romance)               207
(Horror, Thriller)             183
(Drama, Thriller)              174
(Horror,)                      171
(Comedy, Drama)                142
(Music,)                       131
(Documentary, Music)            94
(Comedy, Romance)               93
(Science Fiction, Thriller)     91
(Thriller,)                     84
(Mystery, Thriller)             82
(Crime, Drama)                  76
(Drama, Family)                 66
(Animation, Family)             63
(Horror, Science Fiction)       61
(Comedy, Horror)                57
(Action,)                       56
(Animation,)                    52
(Comedy, Family)                52
(Crime, Thriller)               51
(Romance, Thriller)             48
(Action, Thriller)              48
(Science Fiction,)              43
(Action, Drama)                 39
(Romance,)                      34
(Comedy, Science Fic

In [126]:
genre_big = pd.DataFrame(raw.groupby(['popular_bigram'])['id'].count())
genre_big = genre_big.reset_index()
genre_big.loc[:, 'prop'] = genre_big.loc[:, 'id']/genre_big['id'].sum()
genre_big2 = genre_big[genre_big['prop'] < 0.01]
genre_big.shape, genre_big2.shape

((176, 3), (153, 3))

In [147]:
raw['popular_bigram2'] = raw['popular_bigram']
for i in range(raw.shape[0]):
    if raw.loc[i, 'popular_bigram'] in list(genre_big2['popular_bigram']):
        raw.loc[i, 'popular_bigram2'] = 'Other'